In [1]:
# 라이브러리
import re
import requests
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from html import unescape

import pandas as pd
import numpy as np

from urllib import request

In [ ]:
롯데 : https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID=1016

In [ ]:
cgv : http://www.cgv.co.kr/theaters/?areacode=02&theaterCode=0260&date=20200923

In [ ]:
mega : https://www.megabox.co.kr/theater/time?brchNo=4152

## 1. 롯데시네마 상영 정보 가져오기

URL : https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID=1016

파라미터 : <br>
divisionCode=1 (고정) <br>
detailDivisionCode=1 (지역코드) <br>
cinemaId=1016 (영화관 코드) <br>

In [3]:
# selenium 실행

chrome_path = "/Users/c0re/Documents/lab/sub2/chromedriver"
browser = webdriver.Chrome(chrome_path)

In [60]:
browser.get('https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID=1013')

우선 영화관 지연코드와 영화관 코드를 가져오자.

In [90]:
nav_theater = browser.find_elements_by_xpath("//div[@id='nav']/ul/li")[2]

In [91]:
# 네비게이션에서 영화관 클릭

nav_theater.click()

In [19]:
len(nav_theater.find_elements_by_xpath("div/ul/li"))

9

지역명 가져오기

In [63]:
html = browser.page_source
soup = BeautifulSoup(html, 'lxml')
body = soup.body

In [64]:
li_area_list = body.find('div', attrs={"id":"nav"}).find('li', attrs={"class":"ov"}).find("div").find("ul").find_all("li", recursive=False)

In [99]:
# 지역, 극장명 리스트 만들기

dict_lotte_area_theaters = []

for li_area in li_area_list:
    area = li_area.find('a').text
    # theater_name = [_.find('a').text for _ in li_area.find('div').find('ul').find_all('li', recursive=False)]
    div_theaters = li_area.find('div').find('ul').find_all('li', recursive=False)
    dict_area_theater['area'] = area

    for div_theater in div_theaters:
        dict_theater = {} # 결과값에 담을 딕셔너리
        theater = div_theater.find('a')
        theater_name = theater.text
        url = theater['href']
        dict_theater['area'] = area
        dict_theater['theater'] = theater_name
        dict_theater['url'] = url
        dict_lotte_area_theaters.append(dict_theater)

In [100]:
dict_lotte_area_theaters

[{'area': '스페셜관',
  'theater': '홈',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialIndex'},
 {'area': '스페셜관',
  'theater': '샤롯데',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=300'},
 {'area': '스페셜관',
  'theater': '수퍼플렉스 G',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=941'},
 {'area': '스페셜관',
  'theater': '수퍼 S',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=980'},
 {'area': '스페셜관',
  'theater': '수퍼플렉스',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=940'},
 {'area': '스페셜관',
  'theater': '수퍼 4D',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=930'},
 {'area': '스페셜관',
  'theater': '씨네살롱',
  'url': 'https://www.lottecinema.co.kr/NLCHS/Cinema/SpecialCinema?divisionCode=2&screendivcd=987'},
 {'area': '스페셜관',
  'theater': '씨네패밀리',
  'url': 'h

In [125]:
df_lotte_theater_url = pd.DataFrame(dict_lotte_area_theaters)

In [127]:
# 임시저장
# df_lotte_theater_url.to_csv('lotte_url.csv', mode='w', index=False)

In [85]:
li_area_list[1].find('div').find('ul').find_all('li', recursive=False)

[<li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1013">가산디지털</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1018">가양</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=9010">강동</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1004">건대입구</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1009">김포공항</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1003">노원</a></li>,
 <li><a href="https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&amp;detailDivisionCode=1&amp;cinemaID=1023">도곡</a></li>,
 <li><a href="https://www.lottecinema.co.

In [92]:
elm_area_list = nav_theater.find_elements_by_xpath("div/ul/li")

In [93]:
# 네비게이션에서 지역 클릭

elm_area_list[1].click() # 서울 클릭

In [74]:
# 서울 극장명 개수

len(elm_area_list[1].find_elements_by_xpath("div/ul/li"))

23

In [94]:
elm_theater_list = elm_area_list[1].find_elements_by_xpath("div/ul/li")

In [106]:
# 현재 URL 가져오기
# 롯데시네마 파라미터 수집하기위한 URL 획득

browser.current_url

'https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID=1013#'

In [5]:
def generateLotteTheaterInfo(html):
    lotte_theater_info = [] # 리턴 리스트
    soup = BeautifulSoup(html, "lxml")
    body = soup.body

    div_time_select_wrap = body.find('div', attrs={'id':'timeTable'}).find_all('div', attrs={'class':'time_select_wrap'})
    
    for div_time_select in div_time_select_wrap:
        div_movie_info = div_time_select.find('div', attrs={'class':'list_tit'})
        
        # 등급, 제목
        grade = div_movie_info.find("span").text
        title = div_movie_info.find("p").text
        
        # 스크린
        screen = ",".join([_.text for _ in div_time_select.find_all("ul")[0].find_all("li")])

        # 영화 시간, 좌석, 상영관
        li_theater_info_list = div_time_select.find_all("ul")[1]

        for li_theater_info in li_theater_info_list:
            theater_info = {} # 리턴에 넣을 딕셔너리

            dd_theater_info = [_ for _ in li_theater_info.find_all("dd")]
            start_time = dd_theater_info[0].find("strong").text
            end_time = dd_theater_info[0].find("div").text
            seat_split = dd_theater_info[1].text.replace(" ", "").split("/")

            if len(seat_split) > 1:
                seat_count, seat_total = seat_split

            else :
                seat_count, seat_total = seat_split[0], seat_split[0]
            hall = dd_theater_info[2].text

            theater_info["grade"] = grade
            theater_info["title"] = title
            theater_info["screen"] = screen
            theater_info["start_time"] = start_time
            theater_info["end_time"] = end_time
            theater_info["seat_count"] = seat_count
            theater_info["seat_total"] = seat_total
            theater_info["hall"] = hall
            lotte_theater_info.append(theater_info)
            
    return lotte_theater_info

In [9]:
def wget(method='get', url=None, params=None, data=None):
    try:
        headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
        resp = requests.request(method, url, params=params, data=data, headers=headers)
    except Exception as e:
        print(e)
        resp = ''
    return resp

In [2]:
test_url = 'https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID=1013'

In [4]:
browser.get(test_url)

In [6]:
lotte_theater_info_list = generateLotteTheaterInfo(browser.page_source)

In [7]:
lotte_theater_info_list

[]

## 1. CGV 상영 정보 가져오기

url : http://www.cgv.co.kr/theaters/?areacode=01&theaterCode=0001&date=20200923 <br>
파라미터 : <br>
areacode=01 <br>
theaterCode=0001 <br>
date=20200923

url list 구하기

In [34]:
soup.find('div', attrs={'id':'contents'}).find_all('div', recursive=False)[1]

<div class="wrap-theater">
<h3><img alt="THEATER" src="http://img.cgv.co.kr/R2014/images/title/h3_theater.gif"/></h3>
<div class="sect-theater">
<h4 class="theater-tit"><span>CGV강남</span></h4>
<a class="round inred btn_lease" href="/support/lease/default.aspx"><span style="padding:0 14px;">단체/대관문의</span></a>
<div class="box-thaeteralert" id="divBoxImportantNotice">
<div class="box-innner">
<h5>중요알림</h5>
<div class="info-alert">
<strong>CGV X CAR CINEMA 이용안내</strong>
<p>안녕하세요, CGV입니다.<br/>
<br/>
'CGV X CAR CINEMA'는 서울랜드 피크닉존에 위치한<br/>
자동차극장입니다.<br/>
위치는 네비게이션에서 ' CGV카시네마 서울랜드'로 검색,<br/>
사전 예매는<br/>
<u>CAR CINEMA 선택 -&gt; 일자별 영화 및 시간 선택-&gt; 인원은 차량대수로 '1' 로 선택하셔서 예매</u>하시면 됩니다.<br/>
<br/>
* 10월 휴관 일정 안내 (시설점검)<br/>
 ; 휴관은 매주 월요일<br/>
 10/5(월), 10/12(월), 10/19(월), 10/26(월)<br/>
  추석연휴기간  정상운영합니다.<br/>
</p>
<p>자세한 내용은 카시네마 전용홈페이지 (<a href="http://www.carcinema.co.kr">www.carcinema.co.kr</a>)를 참고부탁드리겠습니다.<br/>
감사합니다.</p>
</div>
<button class="btn-close" id="btn_Important_close" type="button

In [21]:
soup.find_all('div', recursive=)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="No-Cache" http-equiv="Cache-Control"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="width=1024" name="viewport"/>
<meta content="CGV, 시지브이, 영화관, 극장, 영화, 티켓, 박스오피스, 극장, Movie, Theater, Cinema, Cgv, cgv, 예매, 상영작" name="keywords"/>
<meta content="CGV는 선진화된 관람문화와 최고의 서비스로 고객에게 잊을 수 없는 감동을 선사합니다. CGV홈페이지를 통해 영화 예매뿐만 아니라 그 이상의 서비스와 감동을 전달하고, 다양한 즐거움과 특별한 경험을 제공하고자 합니다." name="description"/>
<meta content="영화 그 이상의 감동. CGV" property="og:site_name"/>
<meta content="CGV강남 | 영화 그 이상의 감동. CGV" id="ctl00_og_title" property="og:title"/>
<meta content="http

In [36]:
# selenium 실행

chrome_path = "/Users/c0re/Documents/lab/sub2/chromedriver"
browser = webdriver.Chrome(chrome_path)

browser.get("http://www.cgv.co.kr/theaters/")

In [37]:
html = browser.page_source

In [39]:
soup = BeautifulSoup(html, 'lxml')

In [43]:
li_area_list = soup.find('div', attrs={'class':'sect-city'}).find('ul').find_all('li', recursive=False)

In [56]:
# 지역, 극장명 리스트 만들기

dict_cgv_area_theaters = []

for li_area in li_area_list:
    # 지역명
    area = li_area.find('a').text
    li_theater_list = li_area.find('div', attrs={'class':'area'}).find_all('li')

    for li_theater in li_theater_list:
        dict_theater = {}
        # 극장명, URL
        a_theater = li_theater.find('a')
        theater_name = a_theater.text
        url = a_theater['href']
        dict_theater['area'] = area
        dict_theater['theater'] = theater_name
        dict_theater['url'] = url
        dict_cgv_area_theaters.append(dict_theater)

In [53]:
li_theater_list = li_area_list[0].find('div', attrs={'class':'area'}).find_all('li')

In [57]:
dict_cgv_area_theaters

[{'area': '서울',
  'theater': 'CAR CINEMA',
  'url': '/theaters/?areacode=01&theaterCode=0321&date=20200923'},
 {'area': '서울',
  'theater': 'CGV강남',
  'url': '/theaters/?areacode=01&theaterCode=0056&date=20200923'},
 {'area': '서울',
  'theater': 'CGV강변',
  'url': '/theaters/?areacode=01&theaterCode=0001&date=20200923'},
 {'area': '서울',
  'theater': 'CGV건대입구',
  'url': '/theaters/?areacode=01&theaterCode=0229&date=20200923'},
 {'area': '서울',
  'theater': 'CGV구로',
  'url': '/theaters/?areacode=01&theaterCode=0010&date=20200923'},
 {'area': '서울',
  'theater': 'CGV대학로',
  'url': '/theaters/?areacode=01&theaterCode=0063&date=20200923'},
 {'area': '서울',
  'theater': 'CGV동대문',
  'url': '/theaters/?areacode=01&theaterCode=0252&date=20200923'},
 {'area': '서울',
  'theater': 'CGV등촌',
  'url': '/theaters/?areacode=01&theaterCode=0230&date=20200923'},
 {'area': '서울',
  'theater': 'CGV명동',
  'url': '/theaters/?areacode=01&theaterCode=0009&date=20200923'},
 {'area': '서울',
  'theater': 'CGV명동역 씨네라이브러리',

In [54]:
# 극장명

a_theater = li_theater_list[0].find('a')
print(a_theater.text)
print(a_theater['href'])

CAR CINEMA
/theaters/?areacode=01&theaterCode=0321&date=20200923


In [59]:
# 임시 저장

# df_cgv_theater_url = pd.DataFrame(dict_cgv_area_theaters)
# df_cgv_theater_url.to_csv('cgv_url.csv', mode='w', index=False)

상영시간 구하기

In [ ]:
browser.get('/theaters/?areacode=206%2C04%2C06&theaterCode=0302&date=20200923')

In [8]:
browser.quit()